In [5]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from urllib.request import urlopen
import json

# URL to the JSON file
url = "https://static.bc-edx.com/data/dl-1-2/m14/lms/starter/samples.json"

# Load the JSON data
with urlopen(url) as response:
    data = json.load(response)

# Convert to DataFrame
samples_df = pd.DataFrame(data['samples'])

# Function to create a bar chart
def create_bar_chart(sample_id):
    # Get data for the selected sample
    sample_data = samples_df[samples_df['id'] == sample_id].iloc[0]

    # Extract the top 10 OTUs
    top_10_values = sample_data['sample_values'][:10]
    top_10_ids = sample_data['otu_ids'][:10]
    top_10_labels = sample_data['otu_labels'][:10]

    # Create the bar chart
    fig = go.Figure(go.Bar(
        x=top_10_values[::-1],  # Reverse order for descending
        y=[f"OTU {otu_id}" for otu_id in top_10_ids[::-1]],
        text=top_10_labels[::-1],
        orientation='h'  # Horizontal bar chart
    ))

    # Update layout
    fig.update_layout(
        title=f"Top 10 OTUs for Sample {sample_id}",
        xaxis_title="Sample Values",
        yaxis_title="OTU IDs"
    )
    return fig

# Function to create a bubble chart
def create_bubble_chart(sample_id):
    # Get data for the selected sample
    sample_data = samples_df[samples_df['id'] == sample_id].iloc[0]

    # Create the bubble chart
    fig = go.Figure(data=[go.Scatter(
        x=sample_data['otu_ids'],
        y=sample_data['sample_values'],
        text=sample_data['otu_labels'],
        mode='markers',
        marker=dict(
            size=sample_data['sample_values'],
            color=sample_data['otu_ids'],
            colorscale='Viridis',
            showscale=True
        )
    )])

    # Update layout
    fig.update_layout(
        title=f"Bubble Chart for Sample {sample_id}",
        xaxis_title="OTU ID",
        yaxis_title="Sample Values"
    )
    return fig

# Initial plot for the first sample
sample_ids = samples_df['id'].tolist()
fig_bar = create_bar_chart(sample_ids[0])
fig_bubble = create_bubble_chart(sample_ids[0])

# Add dropdown menu for bar chart
fig_bar.update_layout(
    updatemenus=[{
        'buttons': [
            {
                'label': sample_id,
                'method': 'update',
                'args': [{'x': [samples_df[samples_df['id'] == sample_id].iloc[0]['sample_values'][:10][::-1]],
                          'y': [[f"OTU {otu_id}" for otu_id in samples_df[samples_df['id'] == sample_id].iloc[0]['otu_ids'][:10][::-1]]],
                          'text': [samples_df[samples_df['id'] == sample_id].iloc[0]['otu_labels'][:10][::-1]]},
                         {'title': f'Top 10 OTUs for Sample {sample_id}'}]
            } for sample_id in sample_ids
        ],
        'direction': 'down',
        'showactive': True
    }]
)

# Add dropdown menu for bubble chart
fig_bubble.update_layout(
    updatemenus=[{
        'buttons': [
            {
                'label': sample_id,
                'method': 'update',
                'args': [{'x': [samples_df[samples_df['id'] == sample_id].iloc[0]['otu_ids']],
                          'y': [samples_df[samples_df['id'] == sample_id].iloc[0]['sample_values']],
                          'text': [samples_df[samples_df['id'] == sample_id].iloc[0]['otu_labels']]},
                         {'title': f'Bubble Chart for Sample {sample_id}'}]
            } for sample_id in sample_ids
        ],
        'direction': 'down',
        'showactive': True
    }]
)

# Show the plots
fig_bar.show()
fig_bubble.show()


In [6]:
# Save the bar chart as an HTML file
fig_bar.write_html("bar_chart.html")

# Save the bubble chart as an HTML file
fig_bubble.write_html("bubble_chart.html")
